In [ ]:
import os
#os.chdir(r'C:\Projects\FinSight')
%pwd


'C:\\Projects\\FinSight'

In [54]:
import pandas as pd

In [55]:
data = pd.read_csv("artifacts/data_ingestion/raw_data.csv")


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Ticker    105 non-null    object 
 1   Datetime  105 non-null    object 
 2   Open      105 non-null    float64
 3   High      105 non-null    float64
 4   Low       105 non-null    float64
 5   Close     105 non-null    float64
 6   Volume    105 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 5.9+ KB


In [57]:
data.describe()

,Open,High,Low,Close,Volume
count,105.000000,105.000000,105.000000,105.000000,1.050000e+02
mean,1430.235242,1433.979049,1426.750480,1430.015240,1.172607e+06
std,16.147341,15.891543,16.275826,16.095678,9.293324e+05
min,1400.199951,1406.300049,1395.900024,1400.199951,0.000000e+00
25%,1415.000000,1419.599976,1411.900024,1415.099976,6.619600e+05
50%,1429.000000,1432.900024,1424.699951,1429.000000,9.286360e+05
75%,1445.500000,1448.000000,1441.699951,1443.400024,1.390291e+06
max,1462.099976,1468.199951,1461.099976,1462.199951,4.958397e+06


In [58]:
data.isna().sum()

Ticker      0
Datetime    0
Open        0
High        0
Low         0
Close       0
Volume      0
dtype: int64

In [59]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    data: Path
    all_schemas: dict

In [60]:
from finance_ml.constants import *
from finance_ml.utils.common import read_yaml, create_directories

In [61]:
# src/finance_ml/config/configuration.py

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_Validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_Validation_config = DataValidationConfig(
            root_dir= config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            data= config.data,
            all_schemas= schema  
        )

        return data_Validation_config

In [62]:
import os
from finance_ml import logger

In [63]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True
            data = pd.read_csv(self.config.data)
            all_cols = list(data.columns)
            all_schema = self.config.all_schemas

            for col in all_schema:
                if col not in all_cols: 
                    validation_status = False
                    break

         
            for col, expected_type in all_schema.items():
                if col in data.columns:
                    actual_type = str(data[col].dtype)
                    if actual_type != expected_type:
                        validation_status = False
                        break

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

In [64]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-14 17:29:40,003: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-14 17:29:40,015: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-14 17:29:40,023: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-14 17:29:40,028: INFO: common: created directory at: artifacts]
[2025-06-14 17:29:40,031: INFO: common: created directory at: artifacts/data_validation]
